# Generate a coverage report csv and plots inside docs

In [4]:
%matplotlib inline

from datetime import datetime
import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pathlib
from shapely.geometry import Point
from tqdm import tqdm
import contextily as ctx

## Generate a geopandas data frame with information inside data/

In [5]:
p = pathlib.Path('../data/').glob('*')
valid_files = [x for x in p if x.is_file() if x.suffix=='.xz']
len(valid_files)

451

In [6]:
dfs = []
for file in tqdm(valid_files):
    dfs.append(pd.read_csv(file, dtype={'GEOID20': object}))

odf = pd.concat(dfs)
odf

100%|██████████| 451/451 [00:05<00:00, 77.65it/s] 


,GEOID20,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
0,131050004005010,29.738,9.918,1,1,21,2022,1
1,131050004005010,112.114,16.677,12,5,27,2022,1
2,131050004005010,17.564,8.104,3,2,24,2022,1
3,131050004005009,29.738,9.918,1,1,21,2022,1
4,131050004005009,112.114,16.677,12,5,27,2022,1
...,...,...,...,...,...,...,...,...
1423,132839602001077,28.743,5.776,1,1,18,2022,4
1424,132839602001061,28.743,5.776,1,1,18,2022,4
1425,132839602005062,100.488,47.283,1,1,17,2022,4
1426,132839601002068,22.150,5.297,1,1,37,2022,4


In [7]:
rep_states= sorted(set([s.stem[:2] for s in valid_files]))
rep_states

['01', '11', '13', '24', '42', '51']

In [8]:
# zipfile of U.S. county boundaries
state_url = 'https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_{state}_tabblock20.zip'
gdfs = []
for state in tqdm(rep_states):
    state = gp.read_file(state_url.format(state=state))
    gdfs.append(state)

shape_df = pd.concat(gdfs)
shape_df.to_crs(4326) 
shape_df.head()

100%|██████████| 6/6 [04:52<00:00, 48.67s/it]


,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,01,133,965501,2007,011339655012007,Block 2007,G5040,NaN,NaN,NaN,S,10698,0,+34.2604031,-087.1450636,"POLYGON ((-87.14579 34.26117, -87.14563 34.261..."
1,01,133,965700,3025,011339657003025,Block 3025,G5040,NaN,NaN,NaN,S,27642,0,+34.2430014,-087.6271966,"POLYGON ((-87.62818 34.24378, -87.62790 34.243..."
2,01,133,965601,1006,011339656011006,Block 1006,G5040,NaN,NaN,NaN,S,0,1287383,+34.0254948,-087.2862801,"POLYGON ((-87.30004 34.04209, -87.30001 34.042..."
3,01,133,965900,2065,011339659002065,Block 2065,G5040,NaN,NaN,NaN,S,1151862,0,+34.0175560,-087.4552956,"POLYGON ((-87.46229 34.01465, -87.46224 34.014..."
4,01,063,060102,2001,010630601022001,Block 2001,G5040,NaN,NaN,NaN,S,24414733,86157,+32.8989012,-087.8183437,"POLYGON ((-87.85229 32.85719, -87.85213 32.859..."


In [9]:
shape_df = shape_df.merge(odf, on='GEOID20', how='left')
shape_df

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,INTPTLAT20,INTPTLON20,geometry,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
0,01,133,965501,2007,011339655012007,Block 2007,G5040,NaN,NaN,NaN,...,+34.2604031,-087.1450636,"POLYGON ((-87.14579 34.26117, -87.14563 34.261...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,133,965700,3025,011339657003025,Block 3025,G5040,NaN,NaN,NaN,...,+34.2430014,-087.6271966,"POLYGON ((-87.62818 34.24378, -87.62790 34.243...",64.131,62.988,2.0,2.0,4.0,2022.0,4.0
2,01,133,965700,3025,011339657003025,Block 3025,G5040,NaN,NaN,NaN,...,+34.2430014,-087.6271966,"POLYGON ((-87.62818 34.24378, -87.62790 34.243...",108.346,87.677,1.0,1.0,4.0,2022.0,4.0
3,01,133,965601,1006,011339656011006,Block 1006,G5040,NaN,NaN,NaN,...,+34.0254948,-087.2862801,"POLYGON ((-87.30004 34.04209, -87.30001 34.042...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,133,965900,2065,011339659002065,Block 2065,G5040,NaN,NaN,NaN,...,+34.0175560,-087.4552956,"POLYGON ((-87.46229 34.01465, -87.46224 34.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478775,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",224.321,162.624,115.0,18.0,8.0,2022.0,2.0
5478776,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",398.718,134.073,3.0,3.0,7.0,2022.0,3.0
5478777,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",169.646,127.113,117.0,15.0,7.0,2022.0,3.0
5478778,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",347.473,64.676,18.0,5.0,9.0,2022.0,4.0


In [10]:
shape_df.to_crs("EPSG:4326")

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,INTPTLAT20,INTPTLON20,geometry,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
0,01,133,965501,2007,011339655012007,Block 2007,G5040,NaN,NaN,NaN,...,+34.2604031,-087.1450636,"POLYGON ((-87.14579 34.26117, -87.14563 34.261...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,133,965700,3025,011339657003025,Block 3025,G5040,NaN,NaN,NaN,...,+34.2430014,-087.6271966,"POLYGON ((-87.62818 34.24378, -87.62790 34.243...",64.131,62.988,2.0,2.0,4.0,2022.0,4.0
2,01,133,965700,3025,011339657003025,Block 3025,G5040,NaN,NaN,NaN,...,+34.2430014,-087.6271966,"POLYGON ((-87.62818 34.24378, -87.62790 34.243...",108.346,87.677,1.0,1.0,4.0,2022.0,4.0
3,01,133,965601,1006,011339656011006,Block 1006,G5040,NaN,NaN,NaN,...,+34.0254948,-087.2862801,"POLYGON ((-87.30004 34.04209, -87.30001 34.042...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,133,965900,2065,011339659002065,Block 2065,G5040,NaN,NaN,NaN,...,+34.0175560,-087.4552956,"POLYGON ((-87.46229 34.01465, -87.46224 34.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478775,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",224.321,162.624,115.0,18.0,8.0,2022.0,2.0
5478776,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",398.718,134.073,3.0,3.0,7.0,2022.0,3.0
5478777,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",169.646,127.113,117.0,15.0,7.0,2022.0,3.0
5478778,51,059,491501,1013,510594915011013,Block 1013,G5040,NaN,NaN,NaN,...,+38.8418096,-077.4193256,"POLYGON ((-77.42065 38.84255, -77.41887 38.842...",347.473,64.676,18.0,5.0,9.0,2022.0,4.0


## Generate a plot of coverage based on census block groups

In [11]:
report_dfs = []
shape_df['county'] = shape_df['GEOID20'].apply(lambda x: x[:5])

for county in tqdm(shape_df['county'].unique()):
    pdf = shape_df[shape_df['county']==county]
    total_set = set(pdf['GEOID20'].apply(lambda x: x[:12]))
    empty_set = set(pdf[pdf['avg_d_mbps'].isnull()]['GEOID20'].apply(lambda x: x[:12]).unique())
    
    report_dfs.append(pd.DataFrame(
        {
            'county': county,
            'perc_covered': '%.2f' % ((len(total_set) - len(empty_set))/ len(total_set) * 100),
            'not_covered_count': len(empty_set),
            'total_count': len(total_set),
            'not_covered': ','.join(empty_set),
        },
        index = [0]
    ))
final_report = pd.concat(report_dfs)
final_report= final_report.sort_values(by='county')
final_report.to_csv('../docs/coverage_report.csv', index =False)
final_report

100%|██████████| 451/451 [02:00<00:00,  3.74it/s]


,county,perc_covered,not_covered_count,total_count,not_covered
0,01001,31.11,31,45,"010010211004,010010201001,010010210001,0100102..."
0,01003,15.25,100,118,"010030113004,010030114091,010030114112,0100301..."
0,01005,0.00,22,22,"010059508001,010059503002,010059504003,0100595..."
0,01007,0.00,18,18,"010070100071,010070100012,010070100051,0100701..."
0,01009,0.00,43,43,"010090501072,010090503012,010090504001,0100905..."
...,...,...,...,...,...
0,51800,52.38,40,84,"518000755043,518000757011,518000758013,5180007..."
0,51810,89.81,33,324,"518100454121,518100448063,518100400002,5181004..."
0,51820,88.24,2,17,"518200032001,518200032002"
0,51830,77.78,2,9,"518303703002,518303702003"


## Make plots for each category of columns in ookla in docs

In [13]:
for col in tqdm(['avg_d_mbps', 'avg_u_mbps', 'tests', 'devices', 'avg_lat_ms']):
    ax = shape_df.plot(column='%s' % col, legend=True, scheme='quantiles')
    fig = plt.gcf()
    fig.set_size_inches(17, 11)
    ctx.add_basemap(ax, crs=shape_df.crs.to_string(), source=ctx.providers.Stamen.Toner)
    leg1 = ax.get_legend()
    leg1.set_title("%s" % col)
    plt.savefig('../docs/%s.png' % col, dpi=100)
    plt.clf() # clear plot

100%|██████████| 5/5 [1:23:48<00:00, 1005.71s/it]


<Figure size 1700x1100 with 0 Axes>

<Figure size 1700x1100 with 0 Axes>

<Figure size 1700x1100 with 0 Axes>

<Figure size 1700x1100 with 0 Axes>

<Figure size 1700x1100 with 0 Axes>